In [1]:

import os
import whisper
import pyttsx3
import time

from groq import Groq

import base64
import json
import requests

import numpy as np
import queue
import sys
import wavio

import pyaudio
import wave


import datetime
import tempfile
import threading

def record_audio():
    silence_threshold=200
    silence_duration=1
    chunk_size=1024
    format=pyaudio.paInt16
    channels=1
    rate=44100
    
    p = pyaudio.PyAudio()
    stream = p.open(format=format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk_size)

    frames = []
    silent_timer = None
    stop_recording_flag = threading.Event()

    def stop_recording():
        print("Silence detected. Recording stopped.")
        stop_recording_flag.set()

    try:
        print("Recording started. Please speak into the microphone...")

        while not stop_recording_flag.is_set():
            data = stream.read(chunk_size)
            frames.append(data)
            audio_chunk = np.frombuffer(data, dtype=np.int16)

            if np.max(audio_chunk) < silence_threshold:
                if silent_timer is None:
                    silent_timer = threading.Timer(silence_duration, stop_recording)
                    silent_timer.start()
            else:
                if silent_timer:
                    silent_timer.cancel()
                    silent_timer = None

    except KeyboardInterrupt:
        print("Recording interrupted.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    return frames, format, channels, rate

def save_audio(frames, format, channels, rate):
    temp_dir = tempfile.gettempdir()
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(temp_dir, f"recording_{timestamp}.wav")

    with wave.open(filepath, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(pyaudio.PyAudio().get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

    print(f"Recording saved as: {filepath}")
    return filepath





def run():
    print("In run")
    frames, format, channels, rate = record_audio()
    audio_filepath = save_audio(frames, format, channels, rate)
    
    print("Filepath:", audio_filepath)
    return audio_filepath



#_________________________________________________________________________________________________
#_________________________________________________________________________________________________
#_________________________________________________________________________________________________

def token_count(text):
    tokens = text.split()
    num_tokens = len(tokens)
    return num_tokens

def O_LLM_(query):
    #
    tk_count = token_count(query)
    print("Query Token Count: ",tk_count)
    data = {
    "model": "mistral",
    "prompt": query,
    "stream": False,
    "options": {
    "seed": 123,
    "temperature": 0}
    }
    response = requests.post("http://localhost:11434/api/generate", data=json.dumps(data))
    data = json.loads(response.text)
    answer = data['response']
    print(answer)
    return answer


def O_LLM(query):
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        #model="mixtral-8x7b-32768",
        model="gemma-7b-it",
        temperature = 0,
    )

    response = chat_completion.choices[0].message.content
    return response


#_________________________________________________________________________________________________

def voice(voice_response):
    text = voice_response
    engine = pyttsx3.init()
    engine.setProperty('rate', 190)    # Speed percent (can go over 100)
    engine.setProperty('volume', 0.9)  # Volume 0-1
    engine.say(text)
    engine.runAndWait()

#_________________________________________________________________________________________________


def audio_to_text(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"]

def Voice_input_text_Not_Using(duration):
    duration = int(duration)
    output_dir = tempfile.mkdtemp()
    audio_path = os.path.join(output_dir, "recorded_audio.wav")
    record_audio(duration, audio_path)
    
    if os.path.exists(audio_path):
        #print(f"Audio recorded successfully. Saved at: {audio_path}")
        transcribed_text = audio_to_text(audio_path)
        #print("Transcribed Text:")
        #print(transcribed_text)
        
        return transcribed_text
    else:
        print("Failed to record audio.")
        return "File Not saved"
#_________________________________________________________________________________________________

    
#_________________________________________________________________________________________________

#Voice recording with gap

def Voice_input_text():
    #
    print("temp_file_path")
    temp_file_path = run()
    print("Returned Audio File path: ",temp_file_path)
    transcribed_text = audio_to_text(temp_file_path)
    
    return transcribed_text
#________________________________________________________________________________________________________________________
    
def agent_medication(user_response, chat_history):
    print("Medication")
    chat_his = chat_history_str(chat_history)
    #patient = input("Hello")
    
    query = f"""
Consider you are a medicine assistant reminder and call agent serving medical queries over phone. Complete the following conversation playing the role of medicine reminder assistant. If user has any queries, inform them that you will pass it on to the doctor.\n\n I only want the response. Stricitly no comments, no notes or explantion of any type
Patient: Nick, Suffering from Fever, Medicine: Dolo 650.

By using the information provided, you 'Bot:' making a call with Nick and ask him, Did he take the medication? And ask if he has any discomfort or problems. And ask him if they have any questions, as you will be reporting this to the doctor.
Dont add any symbols, notes, other responses or comments as this will be directly send to Human which will create a huge problem.

Call Started

Chat History:
{chat_his}

Human: {user_response}
Your Response:"""
    
    print("****************************************")
    print(query)
    print("****----------------------------------****")
    
    response = O_LLM(query)
    response = clear_output(response)
    
    return response
#_________________________________________________________________________________________________

chat_history = {"profile": "Their Chat"}

def chat_history_str(chat_history):
    output = f"Profile: {chat_history['profile']}\n"
    for key, value in chat_history.items():
        if key != 'profile':
            output += f"{key}: {value}\n"

    return output

#_________________________________________________________________________________________________

def clear_output(original_string):
    if "Bot:" in original_string:
        substring = original_string.split("Bot:")[1]
    else:
        substring = original_string
    #print(substring)
    return substring
#_________________________________________________________________________________________________




In [3]:
import openai 


messages = [ {"role": "system", "content":  
              "You are a Medicine assistant."} ] 
Run = True

First_message = """
Consider yourself a medicine assistant bot; don't tell the patient your medicine assistant bot they know it. Reminder: You are having a phone call conversation with the patient. Details: NICK Suffering from fever.
The medication doctor suggested: 'Dolo 650 and Zyphir 78' 10 AM Today.
Ask the patient whether they have taken the medication or not. If humans say no, tell them to take it now. If taken, ask if he has any questions, and if there are, say you have recorded it and will send it to the doctor. And remind them of the medication if they have already taken it.
Disclaimer: Don't provide any false claims; make this conversation short and simple. Don't add any notes, as this will be sent to the patient directly. 
Don't make any assumptions about humans or add notes; if you didn't understand the human command, ask him to do so again.
Respond
Dont deviate from the objective: Your a medicine reminder

Bot: Hello Nick, How are you?
"""
voice("Hello Nick, How are you?")

message = Voice_input_text()
messages.append({"role": "assistant", "content": First_message})
for i in range(5):
    print(message)
    print(messages)
    if message: 
        messages.append( 
            {"role": "user", "content": message}, 
        ) 
        chat = openai.ChatCompletion.create( 
            model="gpt-3.5-turbo", messages=messages 
        ) 
    #return resp
    resp = chat.choices[0].message.content 
    print(f"\n ChatGPT: \n{resp}") 
    voice(resp)
    
    messages.append({"role": "assistant", "content": resp})
    message = Voice_input_text()
    



temp_file_path
In run
Recording started. Please speak into the microphone...
Silence detected. Recording stopped.
Recording saved as: C:\Users\akhil\AppData\Local\Temp\recording_20240403_204514.wav
Filepath: C:\Users\akhil\AppData\Local\Temp\recording_20240403_204514.wav
Returned Audio File path:  C:\Users\akhil\AppData\Local\Temp\recording_20240403_204514.wav


C:\Users\akhil\anaconda3\envs\OpenAI_Langchain\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hi Marcus, I'm good how are you?
[{'role': 'system', 'content': 'You are a Medicine assistant.'}, {'role': 'assistant', 'content': "\nConsider yourself a medicine assistant bot; don't tell the patient your medicine assistant bot they know it. Reminder: You are having a phone call conversation with the patient. Details: NICK Suffering from fever.\nThe medication doctor suggested: 'Dolo 650 and Zyphir 78' 10 AM Today.\nAsk the patient whether they have taken the medication or not. If humans say no, tell them to take it now. If taken, ask if he has any questions, and if there are, say you have recorded it and will send it to the doctor. And remind them of the medication if they have already taken it.\nDisclaimer: Don't provide any false claims; make this conversation short and simple. Don't add any notes, as this will be sent to the patient directly.\xa0\nDon't make any assumptions about humans or add notes; if you didn't understand the human command, ask him to do so again.\nRespond\nDo

Exception ignored in: <function BSTR.__del__ at 0x0000020A870304C0>
Traceback (most recent call last):
  File "C:\Users\akhil\anaconda3\envs\OpenAI_Langchain\lib\site-packages\comtypes\__init__.py", line 683, in __del__
    def __del__(self, _free=windll.oleaut32.SysFreeString):
KeyboardInterrupt: 


temp_file_path
In run
Recording started. Please speak into the microphone...
Silence detected. Recording stopped.
Recording saved as: C:\Users\akhil\AppData\Local\Temp\recording_20240403_204615.wav
Filepath: C:\Users\akhil\AppData\Local\Temp\recording_20240403_204615.wav
Returned Audio File path:  C:\Users\akhil\AppData\Local\Temp\recording_20240403_204615.wav


KeyboardInterrupt: 

In [ ]:
#Run on local machine or GROQ

if __name__ == "__main__":
    
    print("start")
    v1 = "Hello Nick, I am Marcus, I am your medication reminder assistant"
    voice(v1)
    i = 0
    usr = "(" + str(i) + ") User"
    bt = "(" + str(i) + ") Bot"
    v1 = "Hello Nick, I am Marcus"
    chat_history[bt] = v1
    
    transcribed_text = Voice_input_text()
    chat_history[usr] = transcribed_text
    chat_his = chat_history_str(chat_history)
    voice_response = agent_medication(transcribed_text,chat_history)
    voice(voice_response)
    
    for i in range(10):
        usr = "(" + str(i+1) + ") User"
        bt = "(" + str(i+1) + ") Bot"
        
        transcribed_text = Voice_input_text()
        chat_history[usr] = transcribed_text
        voice_response = agent_medication(transcribed_text, chat_history)
        print(voice_response)
        voice(voice_response)
        
        chat_history[bt] = voice_response